In [1]:
from typing import Optional
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import warnings
import os.path as osp
from math import pi as PI
from ase.io import read
import torch
import torch.nn.functional as F
from torch.nn import Embedding, Sequential, Linear, ModuleList
import random
from sklearn.model_selection import train_test_split
from torch_scatter import scatter
from torch_geometric.data.makedirs import makedirs
from torch_geometric.data import download_url, extract_zip, Dataset
from torch_geometric.nn import radius_graph, MessagePassing

In [53]:
class SchNet(torch.nn.Module):
    r"""The continuous-filter convolutional neural network SchNet from the
    `"SchNet: A Continuous-filter Convolutional Neural Network for Modeling
    Quantum Interactions" <https://arxiv.org/abs/1706.08566>`_ paper that uses
    the interactions blocks of the form

    .. math::
        \mathbf{x}^{\prime}_i = \sum_{j \in \mathcal{N}(i)} \mathbf{x}_j \odot
        h_{\mathbf{\Theta}} ( \exp(-\gamma(\mathbf{e}_{j,i} - \mathbf{\mu}))),

    here :math:`h_{\mathbf{\Theta}}` denotes an MLP and
    :math:`\mathbf{e}_{j,i}` denotes the interatomic distances between atoms.

    .. note::

        For an example of using a pretrained SchNet variant, see
        `examples/qm9_pretrained_schnet.py
        <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
        qm9_pretrained_schnet.py>`_.

    Args:
        hidden_channels (int, optional): Hidden embedding size.
            (default: :obj:`128`)
        num_filters (int, optional): The number of filters to use.
            (default: :obj:`128`)
        num_interactions (int, optional): The number of interaction blocks.
            (default: :obj:`6`)
        num_gaussians (int, optional): The number of gaussians :math:`\mu`.
            (default: :obj:`50`)
        cutoff (float, optional): Cutoff distance for interatomic interactions.
            (default: :obj:`10.0`)
        max_num_neighbors (int, optional): The maximum number of neighbors to
            collect for each node within the :attr:`cutoff` distance.
            (default: :obj:`32`)
        readout (string, optional): Whether to apply :obj:`"add"` or
            :obj:`"mean"` global aggregation. (default: :obj:`"add"`)
        dipole (bool, optional): If set to :obj:`True`, will use the magnitude
            of the dipole moment to make the final prediction, *e.g.*, for
            target 0 of :class:`torch_geometric.datasets.QM9`.
            (default: :obj:`False`)
        mean (float, optional): The mean of the property to predict.
            (default: :obj:`None`)
        std (float, optional): The standard deviation of the property to
            predict. (default: :obj:`None`)
        atomref (torch.Tensor, optional): The reference of single-atom
            properties.
            Expects a vector of shape :obj:`(max_atomic_number, )`.
    """

    url = 'http://www.quantum-machine.org/datasets/trained_schnet_models.zip'
    def __init__(self, hidden_channels: int = 128, num_filters: int = 128,
                 num_interactions: int = 10, num_gaussians: int = 50,
                 cutoff: float = 10.0, max_num_neighbors: int = 32,
                 readout: str = 'mean', dipole: bool = False,
                # mean: Optional[float] = None, std: Optional[float] = None,
                 atomref: Optional[torch.Tensor] = None
            ):
        super(SchNet, self).__init__()

        import ase

        self.hidden_channels = hidden_channels
        self.num_filters = num_filters
        self.num_interactions = num_interactions
        self.num_gaussians = num_gaussians
        self.cutoff = cutoff
        self.max_num_neighbors = max_num_neighbors
        self.readout = readout
        self.dipole = dipole
        self.readout = 'add' if self.dipole else self.readout
        self.mean = None
        self.std = None
        self.scale = None

        atomic_mass = torch.from_numpy(ase.data.atomic_masses)
        self.register_buffer('atomic_mass', atomic_mass)

        self.embedding = Embedding(100, hidden_channels)
        self.distance_expansion = GaussianSmearing(0.0, cutoff, num_gaussians)

        self.interactions = ModuleList()
        for _ in range(num_interactions):
            block = InteractionBlock(hidden_channels, num_gaussians,
                                     num_filters, cutoff)
            self.interactions.append(block)

        self.lin1 = Linear(hidden_channels, hidden_channels // 2)
        self.act = ShiftedSoftplus()
        self.lin2 = Linear(hidden_channels // 2, 1)

        self.register_buffer('initial_atomref', atomref)
        self.atomref = None
        if atomref is not None:
            self.atomref = Embedding(100, 1)
            self.atomref.weight.data.copy_(atomref)

        self.reset_parameters()

    def reset_parameters(self):
        self.embedding.reset_parameters()
        for interaction in self.interactions:
            interaction.reset_parameters()
        torch.nn.init.xavier_uniform_(self.lin1.weight)
        self.lin1.bias.data.fill_(0)
        torch.nn.init.xavier_uniform_(self.lin2.weight)
        self.lin2.bias.data.fill_(0)
        if self.atomref is not None:
            self.atomref.weight.data.copy_(self.initial_atomref)


    def forward(self, data):
        z = data.z.long()
        pos = data.pos
        batch = data.batch

        """"""
        assert z.dim() == 1 and z.dtype == torch.long
        batch = torch.zeros_like(z) if batch is None else batch

        h = self.embedding(z)

        edge_index = radius_graph(pos, r=self.cutoff, batch=batch,
                                  max_num_neighbors=self.max_num_neighbors)
        row, col = edge_index
        edge_weight = (pos[row] - pos[col]).norm(dim=-1)
        edge_attr = self.distance_expansion(edge_weight)

        for interaction in self.interactions:
            h = h + interaction(h, edge_index, edge_weight, edge_attr)

        h = self.lin1(h)
        h = self.act(h)
        h = self.lin2(h)

        if self.dipole:
            # Get center of mass.
            mass = self.atomic_mass[z].view(-1, 1)
            c = scatter(mass * pos, batch, dim=0) / scatter(mass, batch, dim=0)
            h = h * (pos - c[batch])

        if not self.dipole and self.mean is not None and self.std is not None:
            h = h * self.std + self.mean

        if not self.dipole and self.atomref is not None:
            h = h + self.atomref(z)

        out = scatter(h, batch, dim=0, reduce=self.readout)

        if self.dipole:
            out = torch.norm(out, dim=-1, keepdim=True)

        if self.scale is not None:
            out = self.scale * out

        return out


    def __repr__(self):
        return (f'{self.__class__.__name__}('
                f'hidden_channels={self.hidden_channels}, '
                f'num_filters={self.num_filters}, '
                f'num_interactions={self.num_interactions}, '
                f'num_gaussians={self.num_gaussians}, '
                f'cutoff={self.cutoff})')



class InteractionBlock(torch.nn.Module):
    def __init__(self, hidden_channels, num_gaussians, num_filters, cutoff):
        super(InteractionBlock, self).__init__()
        self.mlp = Sequential(
            Linear(num_gaussians, num_filters),
            ShiftedSoftplus(),
            Linear(num_filters, num_filters),
        )
        self.conv = CFConv(hidden_channels, hidden_channels, num_filters,
                           self.mlp, cutoff)
        self.act = ShiftedSoftplus()
        self.lin = Linear(hidden_channels, hidden_channels)

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.mlp[0].weight)
        self.mlp[0].bias.data.fill_(0)
        torch.nn.init.xavier_uniform_(self.mlp[2].weight)
        self.mlp[0].bias.data.fill_(0)
        self.conv.reset_parameters()
        torch.nn.init.xavier_uniform_(self.lin.weight)
        self.lin.bias.data.fill_(0)

    def forward(self, x, edge_index, edge_weight, edge_attr):
        x = self.conv(x, edge_index, edge_weight, edge_attr)
        x = self.act(x)
        x = self.lin(x)
        return x


class CFConv(MessagePassing):
    def __init__(self, in_channels, out_channels, num_filters, nn, cutoff):
        super(CFConv, self).__init__(aggr='add')
        self.lin1 = Linear(in_channels, num_filters, bias=False)
        self.lin2 = Linear(num_filters, out_channels)
        self.nn = nn
        self.cutoff = cutoff

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.lin1.weight)
        torch.nn.init.xavier_uniform_(self.lin2.weight)
        self.lin2.bias.data.fill_(0)

    def forward(self, x, edge_index, edge_weight, edge_attr):
        C = 0.5 * (torch.cos(edge_weight * PI / self.cutoff) + 1.0)
        W = self.nn(edge_attr) * C.view(-1, 1)

        x = self.lin1(x)
        x = self.propagate(edge_index, x=x, W=W)
        x = self.lin2(x)
        return x

    def message(self, x_j, W):
        return x_j * W


class GaussianSmearing(torch.nn.Module):
    def __init__(self, start=0.0, stop=5.0, num_gaussians=50):
        super(GaussianSmearing, self).__init__()
        offset = torch.linspace(start, stop, num_gaussians)
        self.coeff = -0.5 / (offset[1] - offset[0]).item()**2
        self.register_buffer('offset', offset)

    def forward(self, dist):
        dist = dist.view(-1, 1) - self.offset.view(1, -1)
        return torch.exp(self.coeff * torch.pow(dist, 2))


class ShiftedSoftplus(torch.nn.Module):
    def __init__(self):
        super(ShiftedSoftplus, self).__init__()
        self.shift = torch.log(torch.tensor(2.0)).item()

    def forward(self, x):
        return F.softplus(x) - self.shift

In [47]:
targets=pd.read_csv('targets.csv')
label=targets['energy_per_atom']

In [46]:
targets.head()

,Unnamed: 0,energy,energy_per_atom,fermi_level,homo,lumo,fold
0,0,-349.347640,-7.278076,1.263305,0.253529,2.448582,5
1,1,-349.353704,-7.278202,1.251832,0.229737,2.455558,2
2,2,-349.359991,-7.278333,1.253671,0.232639,2.468721,2
3,3,-349.352917,-7.278186,1.253987,0.232242,2.464204,3
4,4,-349.362675,-7.278389,1.253701,0.232852,2.466078,3


In [16]:
targets['fold'].value_counts()

7    435
5    435
3    435
1    435
6    435
4    435
2    435
0    435
Name: fold, dtype: int64

In [31]:
trains=np.where([targets['fold']!=5])[1]

In [48]:
structures=pd.read_csv('structures.csv')
structures

,Unnamed: 0,_id
0,0,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
1,1,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
2,2,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
3,3,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
4,4,Full Formula (W16 Se1 S30 O1)\nReduced Formula...
...,...,...
3475,3475,Full Formula (W36 Se70 S1 O1)\nReduced Formula...
3476,3476,Full Formula (W36 Se70 S1 O1)\nReduced Formula...
3477,3477,Full Formula (W36 Se70 S1 O1)\nReduced Formula...
3478,3478,Full Formula (W36 Se70 S1 O1)\nReduced Formula...


In [49]:
atoms_list=np.load('atoms_list.npy', allow_pickle=True)

In [50]:
atoms_list

array([Structure Summary
Lattice
    abc : 12.76292132 12.76292132 14.202402
 angles : 90.0 90.0 119.99999999999999
 volume : 2003.515085416219
      A : 12.76292132 0.0 7.815035371153756e-16
      B : -6.381460659999997 11.053014089622023 7.815035371153756e-16
      C : 0.0 0.0 14.202402
PeriodicSite: W (-0.0000, 1.8422, 3.5506) [0.0833, 0.1667, 0.2500]
PeriodicSite: W (-1.5954, 4.6054, 3.5506) [0.0833, 0.4167, 0.2500]
PeriodicSite: W (-3.1907, 7.3687, 3.5506) [0.0833, 0.6667, 0.2500]
PeriodicSite: W (-4.7861, 10.1319, 3.5506) [0.0833, 0.9167, 0.2500]
PeriodicSite: W (3.1907, 1.8422, 3.5506) [0.3333, 0.1667, 0.2500]
PeriodicSite: W (1.5954, 4.6054, 3.5506) [0.3333, 0.4167, 0.2500]
PeriodicSite: W (0.0000, 7.3687, 3.5506) [0.3333, 0.6667, 0.2500]
PeriodicSite: W (-1.5954, 10.1319, 3.5506) [0.3333, 0.9167, 0.2500]
PeriodicSite: W (6.3815, 1.8422, 3.5506) [0.5833, 0.1667, 0.2500]
PeriodicSite: W (4.7861, 4.6054, 3.5506) [0.5833, 0.4167, 0.2500]
PeriodicSite: W (3.1907, 7.3687, 3.5506) [0

In [51]:
from torch_geometric.data import Data
import torch
import ase
from pymatgen.io.ase import AseAtomsAdaptor

i=0
data_atoms = []
for atoms in tqdm(atoms_list):
    atoms=AseAtomsAdaptor.get_atoms(atoms)
    # set the atomic numbers, positions, and cell
    atom = torch.Tensor(atoms.get_atomic_numbers())
    positions = torch.Tensor(atoms.get_positions())
    natoms = positions.shape[0]
    
    # put the minimum data in torch geometric data object
    data = Data(
        pos=positions,
        z= atom,
       # natoms=natoms,
    )
    
    # calculate energy
    data.y = label[i]
    i=i+1
    data_atoms.append(data)

100%|██████████| 3480/3480 [00:02<00:00, 1299.88it/s]


In [9]:
from torch_geometric.data import DataLoader
train_dataset, test_dataset = train_test_split(data_atoms, test_size=0.2)
train_loader = DataLoader(train_dataset, batch_size=4)
test_loader = DataLoader(test_dataset, batch_size=4)

In [10]:
model=SchNet()
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
#device='cpu'
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 10
#
loss_func = torch.nn.L1Loss() #define loss
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=256)

In [ ]:
tr_loss = []
ts_loss=[]#set model to training mode
for epoch in range(200):
    model.train() 
    valid_loss = 0

    train_loss=0

     #shuffle the training data each epoch
    for d in tqdm(train_loader): #go over each training point
        data = d.to(device)#send data to device
       
        out = model(data) 
        optimizer.zero_grad() #zero gradients
        #evaluate data point
        loss = loss_func(out, data.y) #L1 error loss
         #add loss value to aggregate loss
        loss.backward() #compute gradients
        optimizer.step() #apply optimization
        train_loss += loss.item()
    
    model.eval()     # Optional when not using Model Specific layer
    for d in tqdm(test_loader):
        data = d.to(device)
        target = model(data)
        loss = loss_func(target, data.y)
        valid_loss += loss.item()
        
    print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, train_loss/len(train_loader)))
    tr_loss.append(train_loss/len(train_loader))
    print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, valid_loss/len(test_loader)))
    ts_loss.append(valid_loss/len(test_loader))
    
    

  0%|          | 0/696 [00:00<?, ?it/s]/miniconda/envs/crystalvae/lib/python3.6/site-packages/torch/nn/modules/loss.py:96: UserWarning:

Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

100%|██████████| 174/174 [00:03<00:00, 50.49it/s]


Epoch: 000, Average loss: 134.42364
Epoch: 000, Average loss: 119.33974


100%|██████████| 174/174 [00:03<00:00, 45.87it/s]


Epoch: 001, Average loss: 127.82084
Epoch: 001, Average loss: 120.88367


100%|██████████| 174/174 [00:03<00:00, 47.35it/s]


...

100%|██████████| 174/174 [00:03<00:00, 54.21it/s]


Epoch: 038, Average loss: 132.59488
Epoch: 038, Average loss: 129.00992


100%|██████████| 174/174 [00:03<00:00, 48.35it/s]


Epoch: 039, Average loss: 132.59497
Epoch: 039, Average loss: 129.00972


 53%|█████▎    | 368/696 [00:11<00:09, 33.03it/s]

In [12]:
print(model)

SchNet(hidden_channels=128, num_filters=128, num_interactions=4, num_gaussians=50, cutoff=10.0)


# Cross-val

In [54]:
from torch_geometric.data import DataLoader
maes=[]
for a in range(7):
    trains=np.where([targets['fold']!=a])[1]
    tests=np.where([targets['fold']==a])[1]
    train_dataset=[data_atoms[i] for i in trains]
    test_dataset=[data_atoms[i] for i in tests]
    train_loader = DataLoader(train_dataset, batch_size=4)
    test_loader = DataLoader(test_dataset, batch_size=4)
    model=SchNet()
    device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
    #device='cpu'
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    epochs = 10
    #
    loss_func = torch.nn.L1Loss() #define loss
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=256)
    tr_loss = []
    ts_loss=[]#set model to training mode
    for epoch in range(100):
        model.train() 
        valid_loss = 0

        train_loss=0

         #shuffle the training data each epoch
        for d in tqdm(train_loader): #go over each training point
            data = d.to(device)#send data to device

            out = model(data) 
            optimizer.zero_grad() #zero gradients
            #evaluate data point
            loss = loss_func(out, data.y) #L1 error loss
             #add loss value to aggregate loss
            loss.backward() #compute gradients
            optimizer.step() #apply optimization
            train_loss += loss.item()

        model.eval()     # Optional when not using Model Specific layer
        for d in tqdm(test_loader):
            data = d.to(device)
            target = model(data)
            loss = loss_func(target, data.y)
            valid_loss += loss.item()

        print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, train_loss/len(train_loader)))
        tr_loss.append(train_loss/len(train_loader))
        
        print('Epoch: {:03d}, Average loss: {:.5f}'.format(epoch, valid_loss/len(test_loader)))
        ts_loss.append(valid_loss/len(test_loader))
    maes.append(valid_loss/len(test_loader))

    

  0%|          | 0/762 [00:00<?, ?it/s]/miniconda/envs/crystalvae/lib/python3.6/site-packages/torch/nn/modules/loss.py:96: UserWarning:

Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

100%|█████████▉| 760/762 [00:30<00:00, 22.41it/s]/miniconda/envs/crystalvae/lib/python3.6/site-packages/torch/nn/modules/loss.py:96: UserWarning:

Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

 92%|█████████▏| 100/109 [00:01<00:00, 67.43it/s]/miniconda/envs/crystalvae/lib/python3.6/site-packages/torch/nn/modules/loss.py:96: UserWarning:

Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please en

Epoch: 000, Average loss: 0.41070
Epoch: 000, Average loss: 0.75743


100%|██████████| 109/109 [00:01<00:00, 54.84it/s]


Epoch: 001, Average loss: 0.13164
Epoch: 001, Average loss: 0.33875


100%|██████████| 109/109 [00:01<00:00, 55.27it/s]


...

Epoch: 097, Average loss: 0.11068
Epoch: 097, Average loss: 0.34512


100%|██████████| 109/109 [00:02<00:00, 46.77it/s]


Epoch: 098, Average loss: 0.10822
Epoch: 098, Average loss: 0.34916


100%|██████████| 109/109 [00:02<00:00, 46.93it/s]

Epoch: 099, Average loss: 0.10737
Epoch: 099, Average loss: 0.37682


In [55]:
maes

[0.32182802721311193,
 0.33955699766755376,
 0.3224277138675845,
 0.29270045722433186,
 0.33688947923242224,
 0.31148186747685347,
 0.3768232881920327]

In [56]:
np.mean(maes)

0.3288154044105558

In [58]:
np.std(maes)

0.02443561907944912